In [1]:
import requests
import random
import string
import time
from utils import *

In [2]:
config = read_json('config.env')

In [3]:
def get_server_time():
    headers = {
            'service-api-key': config['service_api_key']
        }
    res = requests.get(config['endpoint'] + '/v1/time', headers=headers)
    return res.json()

In [4]:
get_server_time()

{'responseTime': 1647614635117, 'statusCode': 1000, 'statusMessage': 'Success'}

In [5]:
from signature_generator import SignatureGenerator

sg = SignatureGenerator()
get_signature = sg.generate

def GET_v1_services_serviceId():
    nonce = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(8))
    timestamp = int(round(time.time() * 1000))


    path = '/v1/services/{}'.format(config['service_id'])

    headers = {
        'service-api-key': config['service_api_key'],
        'nonce': nonce,
        'timestamp': str(timestamp)
    }


    signature = get_signature(method='GET', path=path, nonce=nonce, timestamp=timestamp, secret=config['service_api_secret'])
    headers['signature'] = signature

    res = requests.get(config['endpoint'] + path, headers=headers)
    return res.json()

In [6]:
GET_v1_services_serviceId()

{'responseTime': 1647614637319,
 'statusCode': 1000,
 'statusMessage': 'Success',
 'responseData': {'serviceId': '6c846b97-1921-4cb0-92b0-42155d4fc3ac',
  'name': 'VIBE',
  'description': 'VIBE',
  'category': 'AI'}}

In [9]:
config

{'endpoint': 'https://test-api.blockchain.line.me',
 'service_api_key': '12691c1d-1611-4b6e-92a9-fdb652dc198f',
 'service_api_secret': '5effa8d1-3e56-41f7-96e4-88d0d9c1a7cd',
 'service_id': '6c846b97-1921-4cb0-92b0-42155d4fc3ac',
 'item': {'contract_id': '839271ce'},
 'wallet_addr': 'tlink1h2wuymysqy8kvmx4py2774rf2mqt53uykynwgr',
 'wallet_secret': 'ahJeQl5fk5SrUA2GngCL60PnJFD1aaaXzTR0eXV2DM8='}

In [36]:
def issue_nft(contract_id):
    nonce = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(8))
    timestamp = int(round(time.time() * 1000))
    path = f'/v1/item-tokens/{contract_id}/non-fungibles'
    request_body = {
        'ownerAddress': config['wallet_addr'],
        'ownerSecret': config['wallet_secret'],
        'name': 'test',
        'meta': 'test_meta'
    }
    headers = {
        'service-api-key': config['service_api_key'],
        'nonce': nonce,
        'timestamp': str(timestamp),
        'Content-Type': 'application/json'
    }


    signature = get_signature(method='POST', path=path, nonce=nonce, timestamp=timestamp, secret=config['service_api_secret'], body=request_body)
    headers['signature'] = signature

    res = requests.post(config['endpoint'] + path, headers=headers, json=request_body)
    return res.json()

def get_items(contract_id):
    path = '/v1/item-tokens/{}/non-fungibles'.format(config['item']['contract_id'])
    res = get_api(path)
    return res.json()

In [21]:
# issue_nft(config['item']['contract_id'])

{'responseTime': 1647615416020,
 'statusCode': 1002,
 'statusMessage': 'Accepted',
 'responseData': {'txHash': 'E5DD65AFBBD3F1A89791B879024A08DA2FF61F9EDEDFFFC6B3C0C00D0F0DDC86'}}

In [37]:
get_items(config['item']['contract_id'])

{'responseTime': 1647625099537,
 'statusCode': 1000,
 'statusMessage': 'Success',
 'responseData': [{'tokenType': '10000003',
   'name': 'test',
   'meta': 'test_meta',
   'createdAt': 1647615418030,
   'totalSupply': '0',
   'totalMint': '0',
   'totalBurn': '0'},
  {'tokenType': '10000002',
   'name': 'test',
   'meta': 'test_meta',
   'createdAt': 1647614986805,
   'totalSupply': '0',
   'totalMint': '0',
   'totalBurn': '0'},
  {'tokenType': '10000001',
   'name': 'playlist',
   'meta': '',
   'createdAt': 1647511671462,
   'totalSupply': '0',
   'totalMint': '0',
   'totalBurn': '0'}]}

In [33]:
def get_api(path):
    nonce = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(8))
    timestamp = int(round(time.time() * 1000))
    headers = {
        'service-api-key': config['service_api_key'],
        'nonce': nonce,
        'timestamp': str(timestamp),
        'signature': get_signature(method='GET', path=path, nonce=nonce, timestamp=timestamp, secret=config['service_api_secret']),
    }
    return requests.get(config['endpoint'] + path, headers=headers)

In [34]:
def get_items_detail(contract_id, token_type):
    path = '/v1/item-tokens/{}/non-fungibles/{}'.format(contract_id, token_type)
    detail = get_api(path)

    path = '/v1/item-tokens/{}/non-fungibles/{}/holders'.format(contract_id, token_type)
    holders = get_api(path)
    return detail.json(), holders.json()

In [35]:
get_items_detail(config['item']['contract_id'], '10000002')

({'responseTime': 1647624994738,
  'statusCode': 1000,
  'statusMessage': 'Success',
  'responseData': {'tokenType': '10000002',
   'name': 'test',
   'meta': 'test_meta',
   'createdAt': 1647614986805,
   'totalSupply': '0',
   'totalMint': '0',
   'totalBurn': '0',
   'token': []}},
 {'responseTime': 1647624994933,
  'statusCode': 1000,
  'statusMessage': 'Success',
  'responseData': []})